In [1]:
train_path = "./MotoC_iPad_plus_train_data_7.txt"
images_dir = "./test/"
output_dir = "./test_polyfit/"
#feature_dev = "iPad"

sum_augment_time = 0

# Import Some Packages

In [2]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from PIL import Image
import numpy as np
import time
import os

# Calculate 

In [3]:
def loadTrainData(data_path):
    input_X = []
    vector = []
    with open(data_path, "r") as data_file:
        for line in data_file:
            label, feature = line.split("|features ")
            label = label.replace("|label ", "")
            
            f_r, f_g, f_b = feature.split(" ")
            input_X.append([int(f_r), int(f_g), int(f_b)])
            
            l_r, l_g, l_b, sp = label.split(" ")
            vector.append([int(l_r), int(l_g), int(l_b),])
    return input_X, vector

In [4]:
input_X, vector = loadTrainData(train_path)
poly = PolynomialFeatures(degree=3)

X_ = poly.fit_transform(input_X)
X_ = np.delete(X_,(0),axis=1)

clf = linear_model.LinearRegression()
clf.fit(X_, vector)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

# Augment

In [5]:
def correct_range(pixel_value):
    if pixel_value < 0:
        pixel_value = 0
        
    if pixel_value > 255:
        pixel_value = 255
        
    return pixel_value

In [6]:
count = 0
for filename in os.listdir(images_dir):
    if filename.endswith(".png"):
        start_time = time.time()
        
        image_path = images_dir + filename
        output_image_name = output_dir + filename
        
        img = Image.open(image_path)
        pixels = img.load()
        
        pixel_list = []
        for i in range(img.size[0]):    
            for j in range(img.size[1]):
                pixel_list.append(img.getpixel((i,j)))
        
        pixel_set = set(pixel_list)
        original_value = list(pixel_set)
        
        predict_ = poly.fit_transform(original_value)
        predict_ = np.delete(predict_, (0), axis=1)
        predicted = clf.predict(predict_)
        predicted = [[int(round(x[0])), int(round(x[1])), int(round(x[2]))] for x in predicted]
        
        combined_dict = {}
        i = 0
        for e in original_value:
            combined_dict[e] = predicted[i]
            i += 1
        
        img = Image.open(image_path)
        pixels = img.load()
        for i in range(img.size[0]):    
            for j in range(img.size[1]):
                key = img.getpixel((i,j))
                R = int(combined_dict[key][0])
                G = int(combined_dict[key][1])
                B = int(combined_dict[key][2])

                R = correct_range(R)
                G = correct_range(G)
                B = correct_range(B)

                pixels[i,j] = (R, G, B)
                
        aug_time = time.time() - start_time
        sum_augment_time += aug_time   
        
        img.save(output_image_name)
        
        if count % 1000 == 0:
            print("Finish : ", filename)
            print("--- %s seconds ---" % (aug_time))
        count += 1
        
print("Total Augment Time :  ", sum_augment_time)
print("Total Images Augmented : ", count)
print("Avg augmentation time per image = %s seconds" % str(sum_augment_time/count))

Finish :  07488.png
--- 0.027683258056640625 seconds ---
Finish :  06838.png
--- 0.011700868606567383 seconds ---
Finish :  08225.png
--- 0.01367497444152832 seconds ---
Finish :  08242.png
--- 0.011688947677612305 seconds ---
Finish :  07599.png
--- 0.011740922927856445 seconds ---
Finish :  09381.png
--- 0.010379791259765625 seconds ---
Finish :  06846.png
--- 0.01242685317993164 seconds ---
Finish :  00607.png
--- 0.022137165069580078 seconds ---
Finish :  00660.png
--- 0.011811971664428711 seconds ---
Finish :  00288.png
--- 0.010913848876953125 seconds ---
Total Augment Time :   119.87458229064941
Total Images Augmented :  10000
Avg augmentation time per image = 0.011987458229064942 seconds
